In [2]:
import pymysql
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='test_ocw',
                             charset='utf8',
                             # Selectの結果をdictionary形式で受け取る
                             cursorclass=pymysql.cursors.DictCursor)

In [8]:
import requests
from bs4 import BeautifulSoup

'''
OCWから学院一覧を取得するスクリプト(6個くらいだから必要ない気もする)
gakuinListの各要素は次のような辞書に鳴っている
{
	'name' : 学院名,
	'url' : その学院の授業の一覧のurl,
}
'''
def getGakuinList():
	url = "http://www.ocw.titech.ac.jp/"
	response = requests.get(url)
	soup = BeautifulSoup(response.content,"lxml")
    
	print("Get GakuinList")

	topMainNav = soup.find("ul",id="top-mein-navi")

	gakubus = topMainNav.find_all(class_="gakubuBox")

	gakuinList = []
	for gakubu_div in gakubus:
		gakuin = gakubu_div.find(class_="gakubuHead").span.string
		#if gakuin[-2::] != "学院":
		#	continue
		gakuin_url = url + gakubu_div.parent['href']
		gakuinList.append({'gakuin':gakuin,'gakuin_url':gakuin_url})

	return gakuinList

'''
[{'name': '理学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=1&lang=JA'},
{'name': '工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=2&lang=JA'},
{'name': '物質理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=3&lang=JA'},
{'name': '情報理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=4&lang=JA'},
{'name': '生命理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=5&lang=JA'},
{'name': '環境・社会理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=6&lang=JA'},
{'name': '教養科目群', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0200&GakubuCD=7&GakkaCD=370000&tab=2&focus=100&lang=JA'},
{'name': '類科目', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=0&lang=JA'}]
'''

'''
学院名とurlを渡されたらその学院の授業一覧を持ってくる
'''
def getLectures(name,url):
    urlprefix = "http://www.ocw.titech.ac.jp"
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'lxml')
    
    print("\tGet Lectures:",name)
    
    tables = soup.find_all('table',class_='ranking-list')
    
    LecList = []

    for t in tables:
        table = t.tbody
        for item in table.find_all('tr'):
            code = item.find('td',class_='code').string
            name = item.find('td',class_='course_title').a.string #講義名
            lecture_url = urlprefix + item.find('td',class_='course_title').a['href']
            teachers = [te.string for te in item.find('td',class_='lecturer').find_all('a')]
            #quater = item.find('td',class_='opening_department').a.string	#TODO ちゃんととれてない
            if not name or not code:	# 文字列が空の場合はスキップ
                continue
            if code:
                code = code.strip()
            if name:
                name = name.strip()
            #if quater:
            #    quater = quater.strip()
            #print(name)
            #print(teachers)
            #print(lecture_url)
            #print(quater)

            LecList.append({"code":code,"name":name,"lecture_url":lecture_url})

    return LecList

#print(getGakuinList())
#getLectures('情報理工学院','http://www.ocw.titech.ac.jp/index.php?module=General&action=T0100&GakubuCD=4&lang=JA')

#Q.曜日・時限が複数になった時もリスト構造にするか(今はリスト構造になっていない)
#各教員にaタグが入っている前提で以下の様になっている
#アクセスランキングは必要に感じなかったので空

#時折text内に凄いへんな文字が入る可能性がある(元のOCWの書かれ方のせい)
def fetch_OCW(Gakuin,Lecture):
    response = requests.get(Lecture["lecture_url"])
    soup = BeautifulSoup(response.content, "html.parser")
    print("\t\tSCRAPE Lecture:",Lecture["name"])
    
    OCW = {}

    for key in soup.find(attrs={"class":"gaiyo-data clearfix"}).find_all("dl"): #上部
        value_list = []
        for value in key.dd.strings:
            value = value.strip().replace(" ","").replace("\n","")
            if len(value)>0: value_list.append(value)
        OCW[key.dt.text] = ",".join(value_list)

    for key in soup.find_all(attrs={"class":"cont-sec"}): #下部
        keyString = key.h3.text

        if key.table is not None: #授業計画・学生が身につける力
            tr_list = []
            #print([j.text for j in key.thead.tr.find_all("th")]) #ヘッダ
            for TR in key.tbody.find_all("tr"): #ボディ
                tr_list.append("[{}]".format(
                    ",".join(["\\'{}\\'".format(j.text) for j in TR.find_all("td")])
                     ))
            OCW[keyString] = "[{}]".format(",".join(tr_list))
            ''' リスト構造で保持する場合
            for TR in i.tbody.find_all("tr"): #ボディ
                print([j.text.split() for j in TR.find_all("td")])
            '''
            
        elif key.ul is not None: #関連する科目
            OCW[keyString] = ",".join([j.text for j in key.ul.find_all("li")])
            
        else: #そのほか(文章で書かれた項目)
            OCW[keyString] = "\n".join(key.p.strings)
    
    #詳細ページに記載のなかった項目は適宜
    OCW["講義名"] = Lecture["name"]
    OCW["URL"] = Lecture["lecture_url"]
    if Gakuin["gakuin"][-2::] != "学院":
        OCW["学院"] = "その他"
    else:
        OCW["学院"] = Gakuin["gakuin"]
        
    return OCW

#データベースのカラム情報
#科目コードをキーとして持たせたい気持ち
column = {# TOP側 #
          "科目コード":"LectureCode", # キー
          "講義名":"LectureName",
          "担当教員名":"Professor",
          "開講元":"Department",
          "曜日・時限(講義室)":"DateRoom",
          "URL":"URL",
         #"講義室":"Room",
          "単位数":"Credit",
          "開講クォーター":"Quarter",
          "使用言語":"Language",
          # シラバス側 #
          "授業計画・課題":"LecturePlan",
          "成績評価の基準及び方法":"AssessStyle",
          "履修の条件(知識・技能・履修済科目等)":"CourseCond",
          # 検索用 #
          "学院":"Gakuin"}

#TABLEをつくる　データベースの構造が完全になったら不要
def createTable(column):
    with connection.cursor() as cursor:
        KEY_COLUMN = "科目コード"
        KEY_LENGTH = 10
        
        sub_column = []
        for k in column:
            if column[k] == KEY_COLUMN:
                #key処理そのいち
                sub_column.append(column[k]+" TEXT NOT NULL")
            else:
                sub_column.append(column[k]+" TEXT")
        
        sub_column.append("PRIMARY KEY({}({}))".format(column[KEY_COLUMN],KEY_LENGTH)) #key処理そのに
        sql = "CREATE TABLE lecture({});".format(",".join(sub_column))
        #print(sql)
        cursor.execute(sql)
        
#デバッグ用　TABLEを削除する
def dropTable():
    print("本当にOCW Tableを削除しますか？(する：y，しない：otherwise)")
    if input()=="y":
        with connection.cursor() as cursor:
            sql = "DROP TABLE lecture;"
            cursor.execute(sql)
    else:
        print("OCW Tableの削除を中止しました")
        
#講義情報をデータベースに格納する
def insertLecture(column,LectureData):
    with connection.cursor() as cursor:
        sql = "INSERT INTO lecture ({}) ".format(",".join(map(lambda x:column[x],column)))
        sql += "VALUES ({}) ".format(",".join(map(lambda x:"\'{}\'".format(LectureData[x]),column)))
        sql += "ON DUPLICATE KEY UPDATE {};".format(",".join(["{} = VALUES({})".format(column[k],column[k]) for k in list(filter(lambda x:x!="科目コード",column))]))
        cursor.execute(sql)
        print("\t\tUPDATE Lecture:",LectureData["講義名"])

In [18]:
#"科目コード":"LectureCode", # キー
#学院":"Gakuin"

def createLforGtable(column):
    with connection.cursor() as cursor:
        sub_column = []
        KEY_LENGTH = 20
        sub_column.append("{} NVARCHAR({}) NOT NULL".format(column["科目コード"],KEY_LENGTH))
        sub_column.append("{} NVARCHAR({}) NOT NULL".format(column["学院"],KEY_LENGTH))
        sub_column.append("PRIMARY KEY({},{})".format(column["科目コード"],column["学院"]))
        sql = "CREATE TABLE LforG({});".format(",".join(sub_column))
        #print(sql)
        cursor.execute(sql)
        
def dropLforGtable():
    print("本当にOCW LforG Tableを削除しますか？(する：y，しない：otherwise)")
    if input()=="y":
        with connection.cursor() as cursor:
            sql = "DROP TABLE LforG;"
            cursor.execute(sql)
    else:
        print("OCW Tableの削除を中止しました")
        
def insertLforG(column,code,gakuin):
    with connection.cursor() as cursor:
        sql = "INSERT IGNORE INTO LforG ({},{}) ".format(column["科目コード"],column["学院"])
        sql += "VALUES (\'{}\',\'{}\') ".format(code,gakuin)
        cursor.execute(sql)
        print("\t\tUPDATE LforG:",code,"-",gakuin)

In [9]:
#OCWスクレイピング実行
if __name__=='__main__':
    #limit値　越えて設定した場合，要素数ぶんが最大になる
    Glimit = 1 #頭からいくつ学院数見るか
    Llimit = 1 #頭からいくつ講義詳細見るか
    
    for Gakuin in getGakuinList()[:Glimit]:
        for Lecture in getLectures(Gakuin["gakuin"],Gakuin["gakuin_url"])[:Llimit]:
            OCWData = fetch_OCW(Gakuin,Lecture)
            insertLecture(column,OCWData)
            connection.commit()

Get GakuinList
	Get Lectures: 理学院
		SCRAPE Lecture: 磁気浮上と磁気支持工学
	UPDATE Lecture: 磁気浮上と磁気支持工学


In [19]:
dropLforGtable()
createLforGtable(column)

本当にOCW LforG Tableを削除しますか？(する：y，しない：otherwise)
a
OCW Tableの削除を中止しました


/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (3720, 'NATIONAL/NCHAR/NVARCHAR implies the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using CHAR(x) CHARACTER SET UTF8MB4 in order to be unambiguous.')
  self._do_get_result()


In [6]:
'''
for Gakuin in getGakuinList():
    for Lecture in getLectures(Gakuin["gakuin"],Gakuin["gakuin_url"]):
        insertLforG(column,Lecture["code"],Gakuin["gakuin"])
'''

'\nfor Gakuin in getGakuinList():\n    for Lecture in getLectures(Gakuin["gakuin"],Gakuin["gakuin_url"]):\n        insertLforG(column,Lecture["code"],Gakuin["gakuin"])\n'

In [ ]:
http://www.ocw.titech.ac.jp/index.php?module=General&action=T0300&GakubuCD=2&GakkaCD=321800&KeiCD=18&KougiCD=201807330&Nendo=2018&lang=JA&vid=03

In [ ]:
OCWData = fetch_OCW(Gakuin,Lecture)

In [66]:
dropTable()
createTable(column)

本当にOCW Tableを削除しますか？(する：y，しない：otherwise)
y


In [79]:
insertLecture(column,OCWData)

	UPDATE Lecture: 解析力学


In [5]:
'''
#検索テスト
with connection.cursor() as cursor:
    sql = "SELECT * FROM lecture"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)
'''

'\n#検索テスト\nwith connection.cursor() as cursor:\n    sql = "SELECT * FROM lecture"\n    cursor.execute(sql)\n \n    dbdata = cursor.fetchall()\n    for rows in dbdata:\n        print(rows)\n'

In [4]:
#検索テスト
'''
with connection.cursor() as cursor:
    sql = "SELECT * FROM LforG"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)
'''

'\nwith connection.cursor() as cursor:\n    sql = "SELECT * FROM LforG"\n    cursor.execute(sql)\n \n    dbdata = cursor.fetchall()\n    for rows in dbdata:\n        print(rows)\n'